In [1]:
import warnings
warnings.filterwarnings("ignore")

import streamlit as st
from streamlit_folium import folium_static
from streamlit_echarts import st_pyecharts

from bs4 import BeautifulSoup
import requests
import re
import urllib.request
import plotly.graph_objects as go

import pandas as pd
import numpy as np
import plotly.express as px

import json

import folium

import pyecharts.options as opts

from pyecharts.charts import Calendar
from pyecharts.charts import Bar, Line, Scatter, EffectScatter, Timeline
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts

import datetime

In [2]:
def json_to_df(url, lista_datos_acumulados):
    """Función para extraer datos por municipio por provincia.
    Esta función habrá que ejecutarla 8 veces (1 vez por provincia).

    Parameters
    ----------
    url : string
        _URL del json de cada provincia con los datos municipales.
    lista_datos_acumulados : list
        lista con los datos acumulados que salen en la tabla de la página de la junta.

    Returns
    -------
    DataFrame
        Dataframe con los datos de la provincia con los municipios como índice.
    """
    payload = {}
    headers = {}
    
    data = []
    response = requests.request("GET", url, headers=headers, data = payload,verify=False)
    df = response.json()
    
    columnas = [x['des'] for x in df['measures']]
    indice = [x[0]['des'] for x in df['data']]
    
    for i in range(len(df['data'])):
        aux = [x['val'] for x in df['data'][i] if 'val' in x.keys()]
        tup = tuple(aux)
        data.append(tup)
    data = pd.DataFrame(data,columns=columnas)
    data['indice'] = indice
    data = data[-data['indice'].isin(lista_datos_acumulados)].reset_index(drop=True)
    data = data.set_index(['indice'])
    return data.applymap(lambda x : float(x) if x!='' else np.nan)
    return data

In [3]:
url = "https://www.juntadeandalucia.es/institutodeestadisticaycartografia/intranet/admin/rest/v1.0/consulta/38637"
lista_acumulados = ['Cádiz','Campo de Gibraltar','Bahía de Cádiz-La Janda','Jerez-Costa Noroeste','Sierra de Cádiz']
cadiz_df = json_to_df(url,lista_acumulados)

In [9]:
import re
re.sub(r'(.*) \((.*)\)',r'\2 \1','Barrios (Los)')

'Los Barrios'

In [11]:
## Limpieza de la tabla
cadiz_df.reset_index(inplace=True)

In [15]:
cadiz_df['indice2'] = [re.sub(r'(.*) \((.*)\)',r'\2 \1',x) if '(' in x else x for x in cadiz_df['indice']]

In [17]:
cadiz_df['indice2'] = [re.sub(r'capital (.*)',r'\1',x) if 'capital' in x else x for x in cadiz_df['indice2']]

In [25]:
with open('Geojson/Cadiz.geojson',encoding="utf-8") as f:
        geo = json.load(f, encoding="utf-8")

In [19]:
cadiz_df

,indice,Población,Confirmados PDIA,Confirmados PDIA 14 días,Tasa PDIA 14 días,Confirmados PDIA 7 días,Total confirmados,Curados,Fallecidos,indice2
0,Algeciras,121957.0,2810.0,372.0,305.025542,135.0,2874.0,2386.0,57.0,Algeciras
1,Barrios (Los),23642.0,343.0,36.0,152.271381,21.0,353.0,291.0,10.0,Los Barrios
2,Castellar de la Frontera,3049.0,59.0,6.0,196.785831,5.0,59.0,50.0,1.0,Castellar de la Frontera
3,Jimena de la Frontera,6951.0,78.0,11.0,158.250611,4.0,81.0,64.0,1.0,Jimena de la Frontera
4,Línea de la Concepción (La),63147.0,1174.0,126.0,199.534420,35.0,1192.0,1013.0,8.0,La Línea de la Concepción
5,San Martín del Tesorillo,2753.0,46.0,2.0,72.648020,2.0,46.0,40.0,0.0,San Martín del Tesorillo
6,San Roque,31218.0,603.0,59.0,188.993529,13.0,614.0,524.0,6.0,San Roque
7,Tarifa,18162.0,402.0,49.0,269.794076,12.0,409.0,348.0,10.0,Tarifa
8,Alcalá de los Gazules,5219.0,70.0,11.0,210.768346,3.0,70.0,36.0,0.0,Alcalá de los Gazules
9,Barbate,22518.0,581.0,164.0,728.306244,68.0,582.0,212.0,6.0,Barbate


In [54]:
m = folium.Map(location=[37, -4], zoom_start=8)

## Mediante folium realizamos el mapa

cloropeth = folium.Choropleth(
    geo_data=geo,
    name='choropleth',
    data=cadiz_df,
    columns=['indice2','Total confirmados'],
    key_on='feature.properties.NAMEUNIT',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2).add_to(m)

m

In [55]:
# add labels indicating the name of the community
style_function = "font-size: 15px; font-weight: bold"
cloropeth.geojson.add_child(
    folium.features.GeoJsonTooltip(['NAMEUNIT'], style=style_function, labels=False))

# create a layer control
folium.LayerControl().add_to(m)

# display map
m